VITERBI - 

The Viterbi algorithm is a dynamic programming method used to find the most likely sequence of hidden states in a Hidden Markov Model (HMM), given a sequence of observed events. It efficiently computes the path with the highest probability by keeping track of the best choices at each step.

In [7]:
import math

# --- HMM definition ---

STATES = ['E', 'F', 'I']                  # E=Exon, F=5'UTR, I=Intron
SYMBOLS = ['A', 'C', 'G', 'T']

# start probabilities (we always begin in Exon)
PI = {'E': 1.0, 'F': 0.0, 'I': 0.0}

# transition probabilities, including a synthetic 'Begin' and 'End'
TRANS = {
    'Begin': {'E': 1.0, 'F': 0.0, 'I': 0.0, 'End': 0.0},
    'E':     {'E': 0.9, 'F': 0.1, 'I': 0.0, 'End': 0.0},
    'F':     {'E': 0.0, 'F': 0.0, 'I': 1.0, 'End': 0.0},
    'I':     {'E': 0.0, 'F': 0.0, 'I': 0.9, 'End': 0.1},
}

# emission probabilities per state
EMIT = {
    'E': {'A':0.25, 'C':0.25, 'G':0.25, 'T':0.25},
    'F': {'A':0.05, 'C':0.00, 'G':0.95, 'T':0.00},
    'I': {'A':0.40, 'C':0.10, 'G':0.10, 'T':0.40},
}

Our goal is to calculate the probability of seeing a particular sequence when following a given path of states in a Hidden Markov Model (HMM). Because these probabilities can become extremely small, we compute the natural logarithm of the probability instead.

The total probability is found by multiplying together all the transition and emission probabilities along the chosen path. When working in log-space, this multiplication turns into addition, meaning the overall log probability is simply the sum of the log of each transition and emission probability used in the sequence.

In [11]:
def ln(x: float) -> float:
    """Return natural log, mapping 0 → -∞."""
    return -math.inf if x == 0.0 else math.log(x)

def compute_log_likelihood(state_sequence: str, dna_sequence: str) -> float:
    """
    Compute log P(path, sequence) under our HMM.
    
    Parameters:
      state_sequence: e.g. "EEEEFFIIII..."
      dna_sequence:    same length string over 'A','C','G','T'
    
    Returns:
      Sum of log-transitions + log-emissions + final log-transition to End.
    """
    if len(state_sequence) != len(dna_sequence):
        raise ValueError("Path and observed string must match in length.")
    
    total_log_prob = 0.0
    prev = 'Begin'
    
    for state, nucleotide in zip(state_sequence, dna_sequence):
        # add log transition
        total_log_prob += ln(TRANS[prev][state])
        # add log emission
        total_log_prob += ln(EMIT[state][nucleotide])
        prev = state
    
    total_log_prob += ln(TRANS[prev]['End'])
    return total_log_prob

# run exmample
if __name__ == "__main__":
    path  = "EEEEEEEEEEEEEEEEEEFIIIIIII"
    seq   = "CTTCATGTGAAAGCAGACGTAAGTCA"
    score = compute_log_likelihood(path, seq)
    print(f"Log prob = {score:.2f}")

Log prob = -41.22


In [10]:
import math

# Re‐label your model parameters:
hidden_states = ['E', '5', 'I']
start_prob    = {'E': 1.0, '5': 0.0, 'I': 0.0}
trans_prob    = {
    'E': {'E': 0.9, '5': 0.1, 'I': 0.0, 'End': 0.0},
    '5': {'E': 0.0, '5': 0.0, 'I': 1.0, 'End': 0.0},
    'I': {'E': 0.0, '5': 0.0, 'I': 0.9, 'End': 0.1},
}
emit_prob     = {
    'E': {'A':0.25, 'C':0.25, 'G':0.25, 'T':0.25},
    '5': {'A':0.05, 'C':0.00, 'G':0.95, 'T':0.00},
    'I': {'A':0.40, 'C':0.10, 'G':0.10, 'T':0.40},
}

# Pre‐compute logs (map zeros → –inf)
def safe_log(x):
    return -math.inf if x == 0.0 else math.log(x)

log_start = {s: safe_log(p) for s,p in start_prob.items()}
log_trans = {s:{t: safe_log(p2) for t,p2 in trans_prob[s].items()} for s in trans_prob}
log_emit  = {s:{o: safe_log(e) for o,e in emit_prob[s].items()} for s in emit_prob}


def viterbi_decode(observations):
    """
    Return the most likely hidden‐state path for the given observations.
    
    We work in log‐space:
      δ[t][s] = max log‐prob of any path ending in state s at time t  
      ψ[t][s] = argmax previous state
    """
    T = len(observations)
    
    # δ and ψ tables
    delta = [ { s: None for s in hidden_states } for _ in range(T) ]
    psi   = [ { s: None for s in hidden_states } for _ in range(T) ]
    
    # Initialization (t = 0)
    for s in hidden_states:
        delta[0][s] = log_start[s] + log_emit[s][observations[0]]
        psi[0][s]   = None
    
    # Recursion
    for t in range(1, T):
        obs = observations[t]
        for curr in hidden_states:
            best_prev, best_score = None, -math.inf
            for prev in hidden_states:
                score = delta[t-1][prev] + log_trans[prev].get(curr, -math.inf) + log_emit[curr][obs]
                if score > best_score:
                    best_prev, best_score = prev, score
            delta[t][curr] = best_score
            psi[t][curr]   = best_prev
    
    # Termination: pick best final state
    final_state, final_score = max(delta[T-1].items(), key=lambda x: x[1])
    
    # Path backtracking
    path = [final_state]
    for t in range(T-1, 0, -1):
        path.append( psi[t][path[-1]] )
    path.reverse()
    
    return "".join(path)

# Example
if __name__ == "__main__":
    seq = "CTTCATGTGAAAGCAGACGTAAGTCA"
    result = viterbi_decode(seq)
    print(result)   


EEEEEEEEEEEEEEEEEEEEEEEEEE
